In [124]:
# 本代码由可视化策略环境自动生成 2021年4月30日18:56
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


m1 = M.instruments.v2(
    start_date='2018-01-01',
    end_date='2018-09-01',
    market='CN_STOCK_A',
    instrument_list="""600519.SHA
000858.SZA
000895.SZA""",
    max_count=0
)

m6 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
close_0
open_0
high_0
low_0
industry_sw_level1_0"""
)

m2 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m6.data,
    start_date='',
    end_date='',
    before_start_days=90
)

m15 = M.auto_labeler_on_datasource.v1(
    input_data=m2.data,
    label_expr="""# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/data_history_data.html
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/big_expr.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
group_rank(industry_sw_level1_0,shift(close_0,-5)/shift(open_0,-1)-1)

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 将分数映射到分类，这里使用20个分类
all_wbins(label, 20)

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high_0, -1) == shift(low_0, -1), NaN, label)
""",
    drop_na_label=True,
    cast_label_int=True,
    date_col='date',
    instrument_col='instrument',
    user_functions={}
)


[2019-01-24 19:06:26.652761] INFO: bigquant: instruments.v2 开始运行..
[2019-01-24 19:06:26.659276] INFO: bigquant: 命中缓存
[2019-01-24 19:06:26.660503] INFO: bigquant: instruments.v2 运行完成[0.007759s].
[2019-01-24 19:06:26.663556] INFO: bigquant: input_features.v1 开始运行..
[2019-01-24 19:06:26.668660] INFO: bigquant: 命中缓存
[2019-01-24 19:06:26.669799] INFO: bigquant: input_features.v1 运行完成[0.006249s].
[2019-01-24 19:06:26.675387] INFO: bigquant: general_feature_extractor.v7 开始运行..
[2019-01-24 19:06:26.679937] INFO: bigquant: 命中缓存
[2019-01-24 19:06:26.681620] INFO: bigquant: general_feature_extractor.v7 运行完成[0.00623s].
[2019-01-24 19:06:26.684382] INFO: bigquant: auto_labeler_on_datasource.v1 开始运行..
[2019-01-24 19:06:27.004980] INFO: 自动标注(任意数据源): 开始标注 ..
[2019-01-24 19:06:29.736701] INFO: bigquant: auto_labeler_on_datasource.v1 运行完成[3.05228s].


In [125]:
m15.plot_label_counts()